## SBST Core Usage

This Python notebook file explains the example usage of sbst_core.py

1. Import requirements

In [21]:
# Import requirements
from sbst_core2 import FitnessManager
import random

2. Load the basic information of functions and their branches

In [22]:
file_path = "../benchmark/collatz_step.py" # path to target

source = open(file_path).read()   
fitness_manager = FitnessManager(source)

# Example: pick first function
functions = fitness_manager.get_functions()

func_info = functions[0]
func_name = func_info.name
func_args = func_info.args
func_dims = func_info.args_dim
func_max_const, func_min_const = func_info.max_const, func_info.min_const

print(func_info)
print("Max/Min const:", func_max_const, func_min_const)

Function collatz_steps with 1 arg(s): ['n']
Max/Min const: 20 0


3. List the branches in the function and choose branch

In [23]:
# list branches in the function
branches = func_info.branches
print("Branches:", branches)

# Choose a branch lineno and an outcome to target (True: taken, False: not taken)
branch_info = branches[0] # e.g., Choose the first branch (zero-based index)
target_branch_node = branch_info.node
target_outcome = True   # e.g. try to make branch taken

print(branch_info)

Branches: [Branch If at lineno=2, Branch While at lineno=6, Branch If at lineno=7, Branch If at lineno=12, Branch If at lineno=14, Branch If at lineno=16]
Branch If at lineno=2


4. Create fitness calculator and calculate fitness for initial random input

In [31]:
# make an initial candidate (random ints for each arg)
initial = [random.randint(-20, 20) for _ in func_args]
initial = [42, 0]

print(initial)
fitness = fitness_manager.calculate_fitness(func_info, branch_info, target_outcome, initial)
print("initial fitness:", fitness, "for", initial)

[42, 0]
initial fitness: inf for [42, 0]


5. Attach to simple HC

In [20]:
import itertools
import random

def generate_neighbors(current_inputs: list[int], step_size: int) -> list[list[int]]:
    steps = itertools.product([0, -step_size, step_size], repeat=len(current_inputs))
    neighbors = []

    for step in steps:
        if all(s == 0 for s in step): continue # Skip the inputs themselves
        neighbors.append([input + s for input, s in zip(current_inputs, step)])

    return neighbors

def hill_climb(step: int = 1, max_iter: int = 10000) -> tuple[list[int], float, int]:
    x = [random.randint(func_min_const, func_max_const) for _ in func_args]
    fitness = fitness_manager.calculate_fitness(func_info, branch_info, target_outcome, x)

    for i in range(max_iter):
        neighbors = generate_neighbors(x, step)
        scored = [(n, fitness_manager.calculate_fitness(func_info, branch_info, target_outcome, n)) for n in neighbors]

        best_neighbor, best_neighbor_fitness = min(scored, key=lambda t: t[1])

        if best_neighbor_fitness <= fitness:
            x, fitness = best_neighbor, best_neighbor_fitness

        if fitness == 0:
            break

        if i % 10 == 0:
            print(f'Iteration #{i} best x: {x} (fitness: {fitness})')

    return x, fitness, i

best_x, best_f, iteration_n = hill_climb()
print(f"Found best after {iteration_n} iters. Best:", best_x, "Fitness:", best_f)

Iteration #0 best x: [817, 724] (fitness: 0.5391177806646443)
Iteration #10 best x: [807, 724] (fitness: 0.5344881633686533)
Iteration #20 best x: [797, 724] (fitness: 0.5298120410103968)
Iteration #30 best x: [787, 724] (fitness: 0.5250889464409336)
Iteration #40 best x: [777, 724] (fitness: 0.5203184078187557)
Iteration #50 best x: [767, 724] (fitness: 0.5154999485626506)
Iteration #60 best x: [757, 724] (fitness: 0.5106330873040895)
Iteration #70 best x: [747, 724] (fitness: 0.5057173378391396)
Iteration #80 best x: [737, 724] (fitness: 0.5007522090798908)
Iteration #90 best x: [727, 724] (fitness: 0.4957372050053961)
Iteration #100 best x: [717, 724] (fitness: 0.490671824612118)
Iteration #110 best x: [707, 724] (fitness: 0.48555556186387894)
Iteration #120 best x: [697, 724] (fitness: 0.4803879056413065)
Iteration #130 best x: [687, 724] (fitness: 0.4751683396907731)
Iteration #140 best x: [677, 724] (fitness: 0.4698963425728212)
Iteration #150 best x: [667, 724] (fitness: 0.46457